# Spark Word Count

Counting words in a text corpus is the "Hello, World" of big data.  We will use counting words in a single document as a brief introduction to fundamental Spark concepts.  Concretely, in this example we'll count occurrences of each word in a text file.  The output will look something like [('the', 1024), ('cat', 992), ...]

## 1.0 Loading Data

Note that PySpark, a Python shell for Spark, has already created the Spark System Context.  The System Context is assigned to a Python variable called sc.

The first step in counting words in a file is loading the file from HDFS.  You should have already stored the file in HDFS prior to beginning this notebook.  The code below reads the specified file from HDFS (note hte HDFS protocol in the URL) and builds a Spark RDD.  Each element in the RDD is a single line in the input file.

In [1]:
lines = sc.textFile("hdfs://sandbox.hortonworks.com/user/spark/221baker.txt")

Here we output the number of input lines read and the first ten lines as a basic sanity check

In [2]:
print lines.count()
print lines.take(10)

481
[u'from: the Asimov collection', u'', u'', u'', u'221B BAKER STREET', u'', u'', u'1.                      >THE UNHOLY MAN<', u'', u'  The  preacher was in  fact a thief who  had stolen the original']


## 2.0 Basic Structural Transformation

Python lambda functions are explained at https://docs.python.org/2/tutorial/controlflow.html#lambda-expressions 

The lines RDD is a collection of lines in the input text file.  To count words it is useful to have a collection of words, not lines.  Thus the code below maps a simple string splitting function onto each line.  Note the use of flatMap instead of map.  Map if applied below would return a RDD collection of lists of words instead of the required collection of words.  We have provided a bit of sample code to illustrate this subtle yet important difference.  Note how the wordsByMap RDD contains a list of words, while the words RDD contains just words.

In [3]:
words = lines.flatMap(lambda x: x.split(" "))

wordsByMap = lines.map(lambda x: x.split(" "))

print words.take(4)
print wordsByMap.take(1)

[u'from:', u'the', u'Asimov', u'collection']
[[u'from:', u'the', u'Asimov', u'collection']]


## 3.0 Filtering and Pre-Processing

If you run a word count on the data as is, you'll notice a few things.  First, there are a lot of empty "words" in the document.  Such words are an artifact of the naive splitting process we used to mark word boundaries.  Second, similar words are split because of capitalization (e.g. The and the).  Finally, the count is dominated by common words that don't tell us much about the contents of the document (e.g. the, in, of, etc.)

The following code defines a simple Python dictionary of stop words.  The stop words list is constructed as is for illustrative purposes.  More robust lists are freely available online.  The first pre-processing step we perform is to convert all words to lower case.  Doing so ensures precise word counts.  We then filter out empty (i.e. length <=0) words and words in the stop words list.  Notice how many words are eliminated.

The code below also demonstrates another important Spark technique.  Spark forms closures (https://en.wikipedia.org/wiki/Closure_(computer_programming)) in order to make variables defined in the driver program available to functions running on remote clusters.  Such variables can be quite large, so copying them repeatedly can impact performance.  Spark Broadcast variables allow read only copies of data to be sent to each machine in a cluster.  Please see http://spark.apache.org/docs/latest/programming-guide.html#broadcast-variables and http://spark.apache.org/docs/latest/programming-guide.html#understanding-closures-a-nameclosureslinka. 

In [4]:
sc_dict = {'the', 'to', 'a', 'and', 'his', 'of', 'in', 'with', 'was', '>the', 
             'for', 'as', 'had', 'her', 'on', 'he', 'at', 'by', 'an', 'that', 'him', 'into', 'then', 'who',
          'from', 'when', 'she', 'would'}
stopwords = sc.broadcast(sc_dict)

print "Mixed case words: %d" % (words.count())

words = words.map(lambda x: x.lower())

print "Lower case words: %d" % (words.count())

words = words.filter(lambda x: len(x) > 0 and not x in stopwords.value)

print "Filtered words: %d" % (words.count())


Mixed case words: 4311
Lower case words: 4311
Filtered words: 1881


## 4.0 Counting Words

If this were a "normal" program and words were simply a Python list, calculating a word count is straightforward.  One can easily envision solutions based on simply sorting the list, or using a dictionary structure to count each word.  Such solutions are not scalable to big data size.  Imagine if our document were a collection of documents that contained billions of words.  Such a data set simply cannot be stored in main memory.  Hence we illustrate how to do a distributed word count in Spark.

The first, and perhaps non-intuitive, step we perform is to transform each word into a Python tuple of the form (word, 1).  We then invoke the reduceByKey on the resultant RDD.  The reduceByKey RDD method aggregates all of the values (1's) associated with a key (words) via a function.  We've used a simple anonymous addition function to perform the addition.  For example, if the input document contained "sword" 9 times, the output of the map below would produce 9 instances of ("sword", 1).  The reduceByKey method simply adds up all of the ones to perform the count.

This complexity is required becuase other approaches do not scale well.  In particular, approaches based on the groupByKey method produce correct results but require extensive data movement.  The reduceByKey method performs reduction within partitions before moving data.

In [5]:
counts = words.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)

The 221B in the input file name provides a clue about the contents of the file.  221B Baker Street is the address of Sherlock  Holmes, so we might expect that the document contains terms related to mysteries, investigations, etc.  Notice the presence of words like motive, killer, and weapon in the output beow.  They are much more suggestive of the document's content than would be a list of stop words.

In [6]:
topCounts = counts.sortBy(lambda x: -x[1])
print topCounts.take(10)

[(u'motive:', 24), (u'killer:', 19), (u'sir', 13), (u'weapon:', 11), (u'killed', 8), (u'after', 7), (u'ring', 6), (u'angel', 6), (u'willie', 6), (u'but', 6)]
